In [2]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
from fs.zipfs import ZipFS
from pathlib import Path
from gwp_uncertainties import add_bw_method_with_gwp_uncertainties

# Local files
from consumption_model_ch.import_databases import (
    import_exiobase_3,
    import_consumption_db,
)
from consumption_model_ch.consumption_fus import (
    add_consumption_activities,
    
    add_consumption_categories,
    add_consumption_sectors,
    add_archetypes_consumption,
    get_archetypes_scores_per_month,
    get_archetypes_scores_per_sector,
)
from consumption_model_ch.plot_archetypes import plot_archetypes_scores_yearly, plot_archetypes_scores_per_sector

# Set environment variables for the bentso package
%set_env ENTSOE_API_TOKEN=98eb7887-8d16-43f4-b75c-1e8a80be520e
%set_env BENTSO_DATA_DIR=/Users/akim/Documents/LCA_files/entso-data-cache

import sys
sys.path.append('/Users/akim/PycharmProjects/akula')
# from akula.electricity import replace_ei_with_entso, add_swiss_residual_mix

env: ENTSOE_API_TOKEN=98eb7887-8d16-43f4-b75c-1e8a80be520e
env: BENTSO_DATA_DIR=/Users/akim/Documents/LCA_files/entso-data-cache


In [3]:
def import_all_databases(use_exiobase, year='091011', add_activities=True, add_archetypes=False):

    path_base = Path('/Users/akim/Documents/LCA_files/')

    directory_habe = path_base / 'HABE_2017/'
    fp_ecoinvent_38 = path_base / "ecoinvent_38_cutoff" / "datasets"
    # fp_ecoinvent_38 = "/Users/cmutel/Documents/lca/Ecoinvent/3.8/cutoff/datasets"
    fp_ecoinvent_33 = path_base / "ecoinvent_33_cutoff"/ "datasets"
    fp_exiobase = path_base / "exiobase_381_monetary" / "IOT_2015_pxp"
    # fp_archetypes = path_base / "heia" / "hh_archetypes_weighted_ipcc_091011.csv"

    ei38_name = "ecoinvent 3.8 cutoff"
    ex38_name = "exiobase 3.8.1 monetary"
    co_name = "swiss consumption 1.0"

    if use_exiobase:
        project = "GSA for archetypes with exiobase"
    else:
        project = "GSA for archetypes"
    bd.projects.set_current(project)

    # Import biosphere and ecoinvent databases
    if ei38_name not in bd.databases:
        bi.bw2setup()
        ei = bi.SingleOutputEcospold2Importer(fp_ecoinvent_38, ei38_name)
        ei.apply_strategies()
        assert ei.all_linked
        ei.write_database()

    exclude_databases = [
        'heia',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - {}'.format(ei38_name),
    ]

    # Import exiobase
    if use_exiobase:
        import_exiobase_3(fp_exiobase, ex38_name)
    else:
        exclude_databases.append('exiobase 2.2')

    # Import consumption database
    try:
        del bd.databases["swiss consumption 1.0"]
    except:
        pass
    if co_name not in bd.databases:
        import_consumption_db(
            directory_habe, co_name, fp_ecoinvent_33, exclude_databases, fp_exiobase,
        )
        
    # Modify electricity mixes
#     add_swiss_residual_mix()
#     replace_ei_with_entso()

    # Add uncertainties to GWP values
    method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    if method not in bd.methods:
        add_bw_method_with_gwp_uncertainties()

    # Add functional units
    co = bd.Database(co_name)
    option = 'aggregated'
    if add_activities:
        add_consumption_activities(co_name, year, option=option,)
        add_consumption_categories(co_name)
        add_consumption_sectors(co_name, year)
        if add_archetypes:
            add_archetypes_consumption(co_name, year)

    # LCIA for average consumption
    co_average_act_name = f'ch hh average consumption {option}, years {year}'
    hh_average = [act for act in co if co_average_act_name == act['name']]
    assert len(hh_average) == 1
    demand_act = hh_average[0]
    lca = bc.LCA({demand_act: 1}, method)
    lca.lci()
    lca.lcia()
    print("{:8.3f}  {}".format(lca.score, demand_act['name']))

#     # LCIA for all Swiss consumption sectors
#     sectors = sorted([act for act in co if "sector" in act['name'].lower() and str(year) in act['name']])
#     for demand_act in sectors:
#         lca = bc.LCA({demand_act: 1}, method)
#         lca.lci()
#         lca.lcia()
#         print("{:8.3f}  {}".format(lca.score, demand_act['name']))
        

def plot_archetypes(use_exiobase, year):
    # Add archetypes and compute total yearly scores per archetype
    if use_exiobase:
        project = "GSA for archetypes with exiobase"
    else:
        project = "GSA for archetypes"
    bd.projects.set_current(project)
    co_name = "swiss consumption 1.0"
    write_dir = Path("write_files") / project.lower().replace(" ", "_") / "archetype_scores"
    write_dir.mkdir(parents=True, exist_ok=True)

    method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
    fp_archetypes_scores = write_dir / f"monthly_scores_{year}.pickle"
    archetypes_scores_monthly = get_archetypes_scores_per_month(co_name, year, method, fp_archetypes_scores)

    # Compare with Andi's results (reproduce Fig. 3 in Andi's data mining paper, top part)
    num_months_in_year = 12
    archetypes_scores_yearly = {
        archetype: score*num_months_in_year for archetype, score in archetypes_scores_monthly.items()
    }
    fig = plot_archetypes_scores_yearly(archetypes_scores_yearly)
    fig.write_html( write_dir / f"yearly_scores_{year}.html")
    fig.write_image(write_dir / f"yearly_scores_{year}.pdf")
    fig.show()

    # Compare with Andi's contributions per sectors (reproduce Fig. 3 in Andi's data mining paper, bottom part)
    fp_archetypes_scores_sectors = write_dir / f"monthly_scores_{year}.pickle"
    archetypes_scores_per_sector = get_archetypes_scores_per_sector(co_name, year, method, write_dir)
    fig = plot_archetypes_scores_per_sector(archetypes_scores_per_sector, year)
    fig.write_html(write_dir / f"sector_scores_{year}.html")
    fig.write_image(write_dir / f"sector_scores_{year}.pdf")
    fig.show()


if __name__ == "__main__":

#     print("Impacts WITHOUT exiobase")
#     print("------------------------")
#     use_ex = False
#     year = '151617'
#     import_all_databases(use_ex, year, True, False)
#     plot_archetypes(use_ex, year)

    print("\n")
    print("Impacts WITH exiobase")
    print("---------------------")
    use_ex = True
    year = '151617'
    import_all_databases(use_ex, year, True, False)
#     plot_archetypes(use_ex, year)




Impacts WITH exiobase
---------------------
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
--> Consumption_db.xlsx already exists, reading it
Created consumption_db.xlsx file in BW format in 1.21 seconds
Extracted 1 worksheets in 1.28 seconds
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges


/Users/akim/PycharmProjects/consumption_model_ch/consumption_model_ch/strategies/consumption_db.py:248: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246

Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applied 0 strategies in 0.00 seconds
207 datasets
6676 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/28/2022 22:27:11
  Finished: 05/28/2022 22:27:12
  Total time elapsed: 00:00:00
  CPU %: 99.90
  Memory %: 2.07
Created database: swiss consumption 1.0
1756.551  ch hh average consumption aggregated, years 151617


In [ ]:
%%time 
# Backup GSA project
bi.backup_project_directory("GSA for archetypes")
# bi.backup_project_directory("GSA for archetypes with exiobase")
# # Restore GSA project
# fp_gsa_project = path_base / "brightway2-project-GSA-backup.16-November-2021-11-50AM.tar.gz"
# if project not in bd.projects:
#     bi.restore_project_directory(fp_gsa_project)